<a href="https://colab.research.google.com/github/JonathanSum/TorchAudioNotes/blob/main/Representing_text_as_Tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

     |████████████████████████████████| 24.2MB 126kB/s 
     |████████████████████████████████| 1.4MB 35.2MB/s 
     |████████████████████████████████| 20.1MB 8.4MB/s 
     |████████████████████████████████| 50.4MB 61kB/s 
     |████████████████████████████████| 804.1MB 20kB/s 
     |████████████████████████████████| 1.9MB 75kB/s 
     |████████████████████████████████| 7.1MB 3.3MB/s 
     |████████████████████████████████| 17.4MB 196kB/s 
     |████████████████████████████████| 1.9MB 22.0MB/s 
     |████████████████████████████████| 901kB 53.8MB/s 
     |████████████████████████████████| 3.3MB 12.4MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434694 sha256=849e99a599d0e8c9ed3b6a01e46079d116d4b299d93b293641b56a7d6f7da16c
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERR

In [2]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

train.csv: 29.5MB [00:00, 70.5MB/s]
test.csv: 1.86MB [00:00, 11.7MB/s]


In [3]:
next(train_dataset)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [4]:
for i,x in zip(range(5),train_dataset):
    print(i)
    print(f"**{classes[x[0]]}** -> {x[1]}")



0
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
1
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
2
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
3
**Sci/Tech** -> Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic m

In [5]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [6]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [7]:
tokenizer('Happy Sugar Life (Japanese: ハッピーシュガーライフ, Hepburn: Happī Shugā Raifu) is a Japanese psychological thriller manga series written and illustrated by Tomiyaki Kagisora.')

['happy',
 'sugar',
 'life',
 '(',
 'japanese',
 'ハッピーシュガーライフ',
 ',',
 'hepburn',
 'happī',
 'shugā',
 'raifu',
 ')',
 'is',
 'a',
 'japanese',
 'psychological',
 'thriller',
 'manga',
 'series',
 'written',
 'and',
 'illustrated',
 'by',
 'tomiyaki',
 'kagisora',
 '.']

In [8]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))

In [9]:
torchtext.vocab.Vocab??

In [10]:
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

In [11]:
vocab_size = len(vocab)
print(f"Vocab size is {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size is 95812


[283, 2321, 5, 337, 19, 1301, 2357]

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

In [13]:
vectorizer.transform(['like hot. its its ran ran']).toarray()

array([[0, 0, 0, 1, 2, 1, 0, 2, 0]])

In [14]:
len(vocab)

95812

In [15]:
r1 = torch.zeros(vocab_size, dtype=torch.float32)

In [16]:
train_dataset[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [19]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([0., 0., 2.,  ..., 0., 0., 0.])


In [25]:
vocab_size

95812

In [26]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

In [27]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

In [28]:
train_dataset[0]


(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [29]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [30]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.808125
6400: acc=0.83796875
9600: acc=0.8533333333333334
12800: acc=0.86


(0.025780931972999816, 0.8636727078891258)